# LangChain Expression Language (LCEL)

In [2]:
# !pip install python-dotenv
# !pip install openai
# !pip install -U langchain langchain-openai

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
OPENAI_API_KEY = openai.api_key

In [6]:
# !pip install langchain_community

In [1]:
# !pip install pydantic==1.10.8

In [7]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [12]:
from openai import OpenAI
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
# client = OpenAI(
#     # This cannot be omitted
#     api_key= OPENAI_API_KEY
# )
model = ChatOpenAI(api_key= OPENAI_API_KEY)

output_parser = StrOutputParser()

In [13]:
chain = prompt | model | output_parser

In [14]:
chain.invoke({"topic": "bears"})

'Why did the bear sit on the stove? \n\nBecause he wanted to be a hot cross bear!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [15]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [20]:
# !pip install docarray

In [21]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(api_key= OPENAI_API_KEY)
)
retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [22]:
retriever.get_relevant_documents("where did harrison work?")

<ipython-input-22-265df7017aee>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [23]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [24]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

we want the first input to be the user question.

We then want context, and then pass into prompt, then model, and get an answer

In [25]:
# runnable map is one way
from langchain.schema.runnable import RunnableMap

In [26]:
# create a runnable object
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),  # call the retriever
    "question": lambda x: x["question"] # using another lambda
}) | prompt | model | output_parser

In [27]:
# get only one response, which is the correct one.
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [28]:
# get context and questions.
# get prompt, model, and then return the chat message. (string)
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [29]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

The idea is a prompt, and a model. And then the function.

When the model is called, parameters are binded and then function.

In [40]:
# need new syntex

# bind parameters

In [30]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [38]:
# prompt = openai.chat.completions.from_messages(
#     [
#         (
#             "system",
#             "Write out the following equation using algebraic symbols then solve it. Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
#         ),
#         ("human", "{equation_statement}"),
#     ]
# )
# model = ChatOpenAI(temperature=0)
# runnable = (
#     {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
# )

# print(runnable.invoke("x raised to the third plus seven equals 12"))

In [39]:
# prompt = openai.chat.completions.create(
#     [
#         ("human", "{input}")
#     ]
# )
# model = ChatOpenAI(temperature=0).bind(functions=functions)

In [41]:
runnable = prompt | model

In [43]:
runnable.invoke({"input": "what is the weather in sf"})

In [ ]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [ ]:
model = model.bind(functions=functions)

In [ ]:
runnable = prompt | model

In [ ]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

## Fallbacks
### (this is actuallly from an older model)

# not only individual componehts, but entire sentences

In [44]:
from langchain.llms import OpenAI
import json

**Note**: Due to the deprecation of OpenAI's model `text-davinci-001` on 4 January 2024, you'll be using OpenAI's recommended replacement model `gpt-3.5-turbo-instruct` instead.

In [66]:
simple_model = OpenAI(
    api_key= OPENAI_API_KEY,
    # temperature=0,
    # max_tokens=1000,
    model="gpt-3.5-turbo-instruct"
    # model = "text-davinci-001" # old
)
simple_chain = simple_model | json.loads

# the result would not be json format

In [75]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [82]:
challenge2 = "write a short story of 100 words in english. be as creative and realistic as possible. A story about everyday life. write this story in a json blob"

In [86]:
challenge3 = "give me a list of gracery list and a short explaination why I need each item. Write in json format"

In [68]:
simple_model.invoke(challenge)

'\n\n{\n  "title": "The Sun and the Moon",\n  "author": "Emily Smith",\n  "first_line": "The sun and the moon, they dance in the sky"\n}\n\n{\n  "title": "Autumn Leaves",\n  "author": "Michael Johnson",\n  "first_line": "Golden leaves falling from the trees"\n}\n\n{\n  "title": "A New Day",\n  "author": "Sarah Lee",\n  "first_line": "As the sun rises, a new day begins" \n}'

In [83]:
simple_model.invoke(challenge2)

'\n\n{\n  "title": "A Day in the Life",\n  "characters": ["Emma", "Mark"],\n  "setting": "Suburban neighborhood",\n  "mainEvent": "Emma and Mark\'s daily routine",\n  "mood": "content",\n  "story": "Emma and Mark woke up to the sound of their alarm clock at 6am. They went for their morning jog around their suburban neighborhood, enjoying the fresh air and each other\'s company. After their run, they had a quick breakfast before heading off to work. Emma worked at a law firm while Mark was a teacher. Despite their busy schedules, they always made time for a lunch date at their favorite cafe. After work, they went grocery shopping and cooked dinner together. As they snuggled on the couch, watching their favorite show, Emma thought to herself, \'This is our everyday life, and I wouldn\'t want it any other way.\'" \n}'

In [87]:
simple_model.invoke(challenge3)

'\n\n\n{\n  "Grocery List": [\n    {\n      "Item": "Milk",\n      "Explanation": "Milk is a good source of calcium and essential for strong bones and teeth."\n    },\n    {\n      "Item": "Eggs",\n      "Explanation": "Eggs are a versatile and protein-rich ingredient that can be used for various dishes."\n    },\n    {\n      "Item": "Bread",\n      "Explanation": "Bread is a staple food and can be used for making sandwiches, toast, and various other dishes."\n    },\n    {\n      "Item": "Fresh Vegetables",\n      "Explanation": "Fresh vegetables are a good source of vitamins and minerals and are essential for a balanced diet."\n    },\n    {\n      "Item": "Fresh Fruits",\n      "Explanation": "Fresh fruits are a good source of antioxidants and essential for a healthy immune system."\n    },\n    {\n      "Item": "Meat",\n      "Explanation": "Meat is a good source of protein and iron and can be used for various dishes."\n    },\n    {\n      "Item": "Pasta",\n      "Explanation": "

**Note**: The next line is expected to fail.

In [51]:
simple_chain.invoke(challenge)

JSONDecodeError: Extra data: line 3 column 2 (char 3)

In [80]:
model = ChatOpenAI(api_key= OPENAI_API_KEY)
chain = model | StrOutputParser() | json.loads

In [54]:
chain.invoke(challenge)


{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'first_line': 'A rose by any other name would smell as sweet.'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'first_line': 'Two roads diverged in a yellow wood,'},
 'poem3': {'title': 'I Wandered Lonely as a Cloud',
  'author': 'William Wordsworth',
  'first_line': 'I wandered lonely as a cloud'}}

In [84]:
chain.invoke(challenge2)


{'title': 'The Daily Commute',
 'word_count': 100,
 'story': 'Every morning, Sarah rushed to catch the crowded bus to work. She squeezed in next to a man in a suit and a woman with a crying baby. The bus jolted and swayed as it made its way through traffic. Sarah checked her phone, scrolling through emails and notifications. As the bus neared her stop, she gathered her things and stood up, ready to face another day at the office. She smiled at the familiar faces around her, knowing they were all in the same boat. The daily grind may be tiring, but it brought a sense of camaraderie.'}

In [88]:
chain.invoke(challenge3)


{'grocery_list': {'eggs': 'to use for baking and cooking recipes',
  'milk': 'for drinking and as an ingredient in various recipes',
  'bread': 'for making sandwiches and toast',
  'chicken': 'to cook for meals throughout the week',
  'vegetables': 'to incorporate into meals for added nutrition',
  'fruits': 'for snacking and adding to dishes for a natural sweetness',
  'rice': 'to use as a side dish or base for meals',
  'pasta': 'for making quick and easy meals',
  'cereal': 'for breakfast or a quick snack',
  'cheese': 'for adding flavor to dishes or as a snack on its own'}}

In [55]:
final_chain = simple_chain.with_fallbacks([chain])

In [56]:
final_chain.invoke(challenge)

{'poem1': {'title': 'The Rose',
  'author': 'Emily Dickinson',
  'firstLine': 'A rose by any other name would smell as sweet.'},
 'poem2': {'title': 'The Road Not Taken',
  'author': 'Robert Frost',
  'firstLine': 'Two roads diverged in a yellow wood,'},
 'poem3': {'title': 'Hope is the Thing with Feathers',
  'author': 'Emily Dickinson',
  'firstLine': 'Hope is the thing with feathers'}}

In [85]:
chain.invoke(challenge2)


{'story': "Every morning, Sarah would wake up to the sound of her alarm clock blaring. She would groggily make her way to the kitchen, brewing a strong cup of coffee to kickstart her day. As she sat at her desk, she would tackle her never-ending to-do list, checking off tasks one by one. In the afternoon, she would take a break and go for a walk in the park, enjoying the fresh air and sunshine. In the evening, she would cook a simple dinner and relax on the couch with a good book. This was Sarah's routine, but she found comfort in its familiarity."}

In [89]:
chain.invoke(challenge3)


{'grocery_list': [{'item': 'eggs',
   'reason': 'a versatile ingredient used in many recipes for breakfast, baking, and cooking'},
  {'item': 'milk',
   'reason': 'a staple for making beverages, baking, and cooking'},
  {'item': 'bread',
   'reason': 'a quick and easy option for making sandwiches or toast'},
  {'item': 'chicken', 'reason': 'a lean protein source for preparing meals'},
  {'item': 'vegetables',
   'reason': 'essential for a balanced diet and adding nutrients to meals'},
  {'item': 'fruits',
   'reason': 'a healthy snack option and source of vitamins and minerals'},
  {'item': 'rice',
   'reason': 'a versatile staple for making side dishes or main courses'},
  {'item': 'pasta',
   'reason': 'a quick and easy option for making a variety of dishes'},
  {'item': 'cereal',
   'reason': 'a convenient breakfast option for a quick and filling meal'},
  {'item': 'snacks',
   'reason': 'to have on hand for quick and satisfying munchies'}]}

## Interface

In [58]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI(api_key= OPENAI_API_KEY)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [71]:
chain.invoke({"topic": "food"})

'Why did the tomato turn red? Because it saw the salad dressing!'

In [72]:
chain.batch([{"topic": "bears"}, {"topic": "food"}])

['Why did the bear bring a backpack to the picnic? \n\nIn case he wanted to go for a bear hike!',
 'Why did the tomato turn red? Because it saw the salad dressing!']

In [74]:
response = await chain.ainvoke({"topic": "food"})
response

# one thing at a time

'Why did the tomato turn red? Because it saw the salad dressing!'

In [73]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why don't bears wear shoes? \nBecause they have bear feet!"

In [59]:
chain.invoke({"topic": "bears"})

"Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!"

In [60]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why did the bear break up with the circus?\n\nBecause he couldn't bear being in a cage anymore!",
 'Why did the frog take the bus to work? Because his car got toad away!']

In [61]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 do
 bears
 have
 hairy
 coats
?

F
ur
 protection
!



In [62]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear bring a flashlight to the party? \nBecause he heard it was going to be a "beary" good time!'